In [2]:
import pandas as pd
import pyspark
import itertools
from sklearn.utils import shuffle
import numpy as np
frac = 0.01

In [2]:
postulaciones = pd.read_csv('Sources/fiuba_4_postulaciones_concat.csv')
test = pd.DataFrame()
postulaciones = postulaciones.drop_duplicates(['idaviso','idpostulante'])
#postulantes = pd.concat([pd.read_csv('Sources/test_final_100k.csv'), pd.read_csv('Sources/fiuba_4_postulaciones_concat.csv')])
postulantes = pd.read_csv('Sources/test_final_100k.csv')
postulantes = postulantes.drop_duplicates('idpostulante')['idpostulante']
#avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv')['idaviso']
avisos = pd.read_csv('Sources/test_final_100k.csv').drop_duplicates('idaviso')['idaviso']

In [3]:
print('postulantes ',postulantes.sample(frac=frac).shape[0])
print('avisos',avisos.sample(frac=frac).shape[0])

postulantes  646
avisos 141


In [4]:
i = 0
print(postulaciones.shape[0],2*postulaciones.shape[0])
while test.shape[0] < 2*postulaciones.shape[0]:
    while test.shape[0] < 2*postulaciones.shape[0]:
        aux = []
        i = i + 1
        postulantes_sample = postulantes.sample(frac=frac)
        avisos_sample = avisos.sample(frac=frac)
        for x in itertools.product(avisos_sample,postulantes_sample):
            aux.append(x)
        aux = pd.DataFrame.from_records(aux)
        aux.rename({0:'idaviso',1:'idpostulante'}, axis=1, inplace=True)
        test = pd.concat([test,aux])
    print('Antes de la poda:', test.shape[0])
    test = test.drop_duplicates(['idaviso','idpostulante'])
    print('Despues de la poda:', test.shape[0])
print(i, test.shape[0],postulaciones.shape[0])

6603752 13207504
Antes de la poda: 13298556
Despues de la poda: 13202227
Antes de la poda: 13293313
Despues de la poda: 13291913
147 13291913 6603752


In [5]:
postulaciones.drop(['Unnamed: 0','fechapostulacion'], axis=1, inplace=True)

In [6]:
postulaciones['sepostulo'] = 1

In [7]:
postulaciones.shape

(6603752, 3)

In [8]:
postulaciones = pd.concat([postulaciones,test])

In [9]:
postulaciones.shape

(19895665, 3)

In [10]:
postulaciones = postulaciones.drop_duplicates(['idaviso','idpostulante'])
postulaciones['sepostulo'].fillna(0,inplace=True)

In [11]:
postulaciones.shape

(19883189, 3)

In [12]:
test = pd.read_csv('Sources/test_final_100k.csv').drop('id', axis=1)
postualciones = pd.concat([test,postulaciones])
postualciones = postulaciones.drop_duplicates(['idaviso','idpostulante'])
postulaciones = postulaciones[100000:]
postulaciones.shape

(19783189, 3)

In [13]:
postualciones.shape

(19883189, 3)

In [14]:
postulaciones = shuffle(postulaciones)

In [15]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
11941,1112428133,aRAV90,0.0
82495,1112410032,10eV0Q,0.0
952,1111667482,EkqWmb,0.0
87673,1112390628,ZL1PGz,0.0
16903,1112300657,eo2AB9,0.0


In [16]:
postulaciones.to_csv('Sources/postulaciones_for_training.csv')

In [2]:
postulaciones = pd.read_csv('Sources/fiuba_4_postulaciones_concat.csv').drop(['Unnamed: 0','fechapostulacion'], axis=1)
visitas = pd.read_csv('Sources/visitas_proc.csv').drop(['Unnamed: 0'],axis=1)

In [3]:
print('Postulaciones', postulaciones.shape)
print('Visitas', visitas.shape)

Postulaciones (8311264, 2)
Visitas (8597915, 3)


In [4]:
visitas.head()

,idpostulante,idaviso,visito
0,0002q,1789742,4
1,0002q,1807692,6
2,0002q,1808090,5
3,0005E,1744005,3
4,0005E,1785430,2


In [5]:
postulaciones['sepostulo'] = 1
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112248724,NjlD,1
1,1112286523,ZaO5,1
2,1112272060,ZaO5,1
3,1112288401,ZaO5,1
4,1112300563,ZaO5,1


In [6]:
ds_train = postulaciones.merge(visitas, on=['idaviso','idpostulante'], how='outer')

In [7]:
print('ds_train', ds_train.shape)

ds_train (15135144, 4)


In [29]:
ds_train.head()

,idaviso,idpostulante,sepostulo,visito
0,1112248724,NjlD,1.0,NaN
1,1112286523,ZaO5,1.0,NaN
2,1112286523,ZaO5,1.0,NaN
3,1112272060,ZaO5,1.0,NaN
4,1112272060,ZaO5,1.0,NaN


In [36]:
ds_train['sepostulo'].fillna(0,inplace=True)

In [31]:
ds_train.loc[ds_train['visito']==0].shape

(0, 4)

In [38]:
np.nanmean(ds_train.loc[ds_train['sepostulo']==1,'visito'])

2.1685270894363606

In [39]:
np.nanmean(ds_train.loc[ds_train['sepostulo']==0,'visito'])

2.126310544734081

In [40]:
ds_train['visito'].fillna(np.nanmean(ds_train.loc[ds_train['sepostulo']==1,'visito']),inplace=True)

In [1]:
ds_train = shuffle(ds_train)

NameError: name 'shuffle' is not defined

In [42]:
ds_train.to_csv('Sources/postulaciones_visitas.csv')